In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 Thinking mode

* [Gemini 2.0 Flash Thinking Mode](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking-mode)
  * Gemini 2.0 Flash Thinking Mode is an experimental model that's trained to generate the "thinking process" the model goes through as part of its response.
  * As a result, Gemini 2.0 Flash Thinking Mode is capable of stronger reasoning capabilities in its responses than the base Gemini 2.0 Flash model.



* [google-genai 0.3.0](https://pypi.org/project/google-genai/)
* [Google Gen AI SDK](https://googleapis.github.io/python-genai/)
* [python-genai : Github](https://github.com/googleapis/python-genai)

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-genai

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [38]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-thinking-exp-1219" # @param {type:"string"}

### Import libraries

In [39]:
import base64

from google import genai
from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)

### Initalize client

* [genai.Client](https://googleapis.github.io/python-genai/genai.html#genai.client.Client)

In [40]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION
)

### Helper function

In [43]:
def generate_gemini_2_0(query:str) -> GenerateContentResponse:

  contents = [
    Content(
      role="user",
      parts=[
        # https://googleapis.github.io/python-genai/genai.html#genai.types.Part
        # Check class methods. (from_bytes, from_code_execution_result, from_executable_code, from_function_call, etc... )
        Part.from_text(query)
      ]
    )
  ]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.SafetySettingDict
  safety_settings = [SafetySetting(category="HARM_CATEGORY_HATE_SPEECH",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_HARASSMENT",threshold="OFF")]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
  generate_content_config = GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = safety_settings,
  )

  # https://googleapis.github.io/python-genai/genai.html#genai.models.AsyncModels.generate_content_stream
  responses = client.models.generate_content(
    model = MODEL_NAME,
    contents = contents,

    config = generate_content_config,
  )

  return responses

### Execution

In [44]:
prompt = """
어떤 섬에는 '진실 부족'과 '거짓 부족'이 살고 있습니다. 진실 부족 사람들은 항상 진실만을 말하고, 거짓 부족 사람들은 항상 거짓만을 말합니다.
당신은 이 섬에 도착하여 세 명의 섬 주민 A, B, C를 만났습니다. 당신은 이들이 어느 부족에 속하는지 알 수 없습니다.

A는 "B와 C는 같은 부족입니다."라고 말했습니다.
B는 "A는 진실 부족입니다."라고 말했습니다.
C는 "A는 거짓 부족입니다."라고 말했습니다.

세 사람 A, B, C는 각각 어느 부족에 속할까요?

"""
responses = generate_gemini_2_0(prompt)
display(Markdown(responses.text))


이 문제는 전형적인 논리 퍼즐입니다. 각 사람의 말이 진실인지 거짓인지 가정하여 모순이 없는 경우를 찾아야 합니다.

**가능한 경우를 나누어 생각해 봅시다.**

**경우 1: A가 진실 부족인 경우**

* A의 말 "B와 C는 같은 부족입니다."는 참입니다. 즉, B와 C는 둘 다 진실 부족이거나 둘 다 거짓 부족입니다.
* B의 말 "A는 진실 부족입니다."는 참입니다. (가정에서 A는 진실 부족이므로) 따라서 B는 진실 부족입니다.
* B가 진실 부족이므로, A의 말에 따라 C도 진실 부족이어야 합니다.
* C의 말 "A는 거짓 부족입니다."는 거짓입니다. (A는 진실 부족이므로) 따라서 C는 거짓 부족이어야 합니다.

이 경우, B는 진실 부족이면서 C도 진실 부족이어야 하지만, C는 거짓 부족이어야 하므로 모순이 발생합니다. 따라서 A는 진실 부족일 수 없습니다.

**경우 2: A가 거짓 부족인 경우**

* A의 말 "B와 C는 같은 부족입니다."는 거짓입니다. 즉, B와 C는 서로 다른 부족입니다.
* B의 말 "A는 진실 부족입니다."는 거짓입니다. (가정에서 A는 거짓 부족이므로) 따라서 B는 거짓 부족입니다.
* C의 말 "A는 거짓 부족입니다."는 참입니다. (가정에서 A는 거짓 부족이므로) 따라서 C는 진실 부족입니다.

이 경우, B는 거짓 부족이고 C는 진실 부족이므로 서로 다른 부족이라는 A의 거짓말과 일치합니다.

**결론**

위의 분석을 통해, **A는 거짓 부족, B는 거짓 부족, C는 진실 부족**이라는 결론을 얻을 수 있습니다.